In [ ]:
import os
import sys
import glob
import shutil
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('..')

In [ ]:
from preprocessing.BatchCt import BatchCt
from dataset import *

### Dirs for Index and dump

In [ ]:
DIR_DATA = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [ ]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

### Create dataset index and Dataset

In [ ]:
ind = FilesIndex(DIR_DATA, dirs=True)

In [ ]:
len(ind.index)

In [ ]:
ds = Dataset(index=ind, batch_class=BatchCt)

In [ ]:
ds.cv_split([0.01, 0.8])

### Create dictionary with paths for load

In [ ]:
dicty = dict(zip(ind.index, glob.glob(DIR_DATA)))

In [ ]:
dicty

### Run preprocess on train (on $\approx$ 30 scans)

In [ ]:
# lazy (write in .future)
args_resize = dict(num_threads=20, num_slices_new=128, num_x_new=256, num_y_new=256)
args_segment = dict(erosion_radius=2, num_threads=20)

pp_lazy = (Preprocessing(ds.train).load(all_patients_paths=dicty, btype='dicom').
           resize(**args_resize).
           segment(**args_segment).dump(path=DIR_DUMP))

In [ ]:
# run itself
BATCH_SIZE = 8
pp_lazy.run(batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
len(os.listdir(DIR_DUMP))